In [1]:
%run SetUp.ipynb

from scipy.optimize import minimize

from matplotlib import pyplot as plt

In [ ]:
nQ = 6

## Define VQE form (Using form for vacuum prep for now)

In [ ]:
# Layers of VQE

def PYP(angle,nq):
    n_sets = int(nq/3)
    mat = sp.sparse.identity(2**nq)
    angle = angle/2

    for mmm in range(n_sets):
        cc,dd,ee = (3*mmm+2)%nq,(3*mmm+3)%nq,(3*mmm+4)%nq
        mat @= sp.sparse.linalg.expm(-1j * angle * pad_op(cc,nq,proj0) @ pad_op(dd,nq,y) @ pad_op(ee,nq,proj0))
        mat @= sp.sparse.linalg.expm(-1j * angle * pad_op(cc,nq,proj1) @ pad_op(dd,nq,y) @ pad_op(ee,nq,proj1))

    for mm in range(n_sets):
        bb,cc,dd = (3*mm+1)%nq,(3*mm+2)%nq,(3*mm+3)%nq
        mat @= sp.sparse.linalg.expm(-1j * angle * pad_op(bb,nq,proj0) @ pad_op(cc,nq,y) @ pad_op(dd,nq,proj0))
        mat @= sp.sparse.linalg.expm(-1j * angle * pad_op(bb,nq,proj1) @ pad_op(cc,nq,y) @ pad_op(dd,nq,proj1))

        
    for m in range(n_sets):
        aa,bb,cc = (3*m)%nq,(3*m+1)%nq,(3*m+2)%nq
        mat @= sp.sparse.linalg.expm(-1j * angle * pad_op(aa,nq,proj0) @ pad_op(bb,nq,y) @ pad_op(cc,nq,proj0))
        mat @= sp.sparse.linalg.expm(-1j * angle * pad_op(aa,nq,proj1) @ pad_op(bb,nq,y) @ pad_op(cc,nq,proj1))
    

    return mat


In [ ]:
def vqe_form(angles,nq):
    it = iter(angles)
    mat = sp.sparse.identity(2**nq)
    for i in range(len(angles)):
        angle = next(it)
        mat @= PYP(angle,nq)
    return mat

## Propagated State Prep by minimizing infidelity

In [ ]:
fake_vac =  sp.sparse.csr_array(ft.reduce(sp.sparse.kron, [[1.,0.]]* nQ).toarray()).transpose()
real_vac = fake_vac #placeholder

In [ ]:
# Generate a state from a set of angles
# Want to minimize the infidelity

num_params = 1

target = np.array([1])  # placeholder
Hf = sp.sparse.csr_matrix(np.real(Hfull(nQ, m, g).to_matrix()))


def infidel(angles):  # test this and see if does what I expect it to do # can check against target and see if get 0
    state = vqe_form(angles,nQ).dot(real_vac)
    return 1 - (target.getH() @ state) ** 2  # need to do abs? nope do np.real when need to remove negative values

In [ ]:
initial_point = np.random.rand(num_params) 

infidel(initial_point)

In [ ]:
state_result = minimize(infidel, initial_point, method='powell',
               options={'xatol': 1e-8, 'disp': True})

NameError: name 'minimize' is not defined